In [1]:
# Import the necessary modules
from ow_ustx import *
import json
import time
from pyfus.io.ustx import PulseProfile, DelayProfile, TxModule, TxArray, Tx7332Registers, print_regs
from pyfus.xdc import Transducer, Element
import numpy as np
import sys
from ow_ustx import *
import json
import time

In [2]:
vid = 1155  # Example VID for demonstration
pid = 22446  # Example PID for demonstration
 
com_port = list_vcp_with_vid_pid(vid, pid)
serial_obj = None
if com_port is None:
    print("No device found")
else:
    print("Device found at port: ", com_port)
    # Select communication port
    serial_obj = UART(com_port, timeout=5)


INFO:UART:Connecting to COM port at COM18 speed 921600


Device found: COM18 - USB Serial Device (COM18)
Device found at port:  COM18


In [3]:
# Initialize the USTx controller object
ustx_ctrl = CTRL_IF(serial_obj)

In [4]:
# enumerate devices
print("Enumerate I2C Devices")
ustx_ctrl.enum_i2c_devices()
print("Enumerate TX7332 Chips on AFE devices")
for afe_device in ustx_ctrl.afe_devices:
    print("Enumerate TX7332 Chips on AFE device")
    rUartPacket = afe_device.enum_tx7332_devices()

Enumerate I2C Devices
Enumerate TX7332 Chips on AFE devices
Enumerate TX7332 Chips on AFE device
Status Packet:
  ID: 2
  Command: 0x31
  Status: 0x0
  Reserved: 0x2
  Data Length: 8
  CRC: 0x3867
Enumerate TX7332 Chips on AFE device
Status Packet:
  ID: 3
  Command: 0x31
  Status: 0x0
  Reserved: 0x2
  Data Length: 8
  CRC: 0x7dc7


In [17]:
# Set the focus and pulse profile
focus = np.array([0, 0, 50]) #set focus #left, front, down 
pulse_profile = PulseProfile(profile=1, frequency=400e3, cycles=3)
afe_dict = {afe.i2c_addr:afe for afe in ustx_ctrl.afe_devices}
# Load Mapping file
arr = Transducer.from_file(R"M2.json")
arr.elements = np.array(arr.elements)[np.argsort([el.pin for el in arr.elements])].tolist()
distances = np.sqrt(np.sum((focus - arr.get_positions(units="mm"))**2, 1))
tof = distances*1e-3 / 1500
delays = tof.max() - tof 
txa = TxArray(i2c_addresses=list(afe_dict.keys()))
array_delay_profile = DelayProfile(1, delays.tolist())
txa.add_delay_profile(array_delay_profile)
txa.add_pulse_profile(pulse_profile)
regs = txa.get_registers(profiles="configured", pack=True) 
for addr, rm in regs.items():
    print(f'I2C: 0x{addr:02x}')
    for i, r in enumerate(rm):
        print(f'MODULE {i}')
        print_regs(r)
    print('')  #calculate register state for 7332s, settings for board (bits, purpose), #change focus!!

INFO:root:Initializing transducer array


I2C: 0x38
MODULE 0
0x0:x00000000
0x1:x00000000
0x6:x00000000
0xB:x00000000
0xC:x00000000
0xF:x00000000
0x14:x00000000
0x15:x00000000
0x16:x00000000
0x18:x02000003
0x19:x00000744
0x1A:x00000000
0x1B:x00000000
0x1E:x00000000
0x1F:x00000000
0x20[+0]:x008B008F
0x20[+1]:x006C0068
0x20[+2]:x00940092
0x20[+3]:x00960096
0x20[+4]:x0071006F
0x20[+5]:x00730073
0x20[+6]:x0071006F
0x20[+7]:x00730073
0x20[+8]:x00940092
0x20[+9]:x00960096
0x20[+10]:x008B008F
0x20[+11]:x00A400A0
0x20[+12]:x0068006C
0x20[+13]:x003C0038
0x20[+14]:x00420043
0x20[+15]:x003F0041
0x120[+0]:xF1D092F2
0x120[+1]:x0007D091
MODULE 1
0x0:x00000000
0x1:x00000000
0x6:x00000000
0xB:x00000000
0xC:x00000000
0xF:x00000000
0x14:x00000000
0x15:x00000000
0x16:x00000000
0x18:x02000003
0x19:x00000744
0x1A:x00000000
0x1B:x00000000
0x1E:x00000000
0x1F:x00000000
0x20[+0]:x00AB00AC
0x20[+1]:x00A700A9
0x20[+2]:x00AB00AC
0x20[+3]:x00A700A9
0x20[+4]:x00420043
0x20[+5]:x003F0041
0x20[+6]:x00000002
0x20[+7]:x003C0038
0x20[+8]:x00A800AB
0x20[+9]:x00A

In [18]:
# Write Registers to Device #series of loops for programming tx chips 
for i2c_addr, module_regs in regs.items():
    afe = afe_dict[i2c_addr] 
    for i in range(len(module_regs)):
        device = afe.tx_devices[i]
        r = module_regs[i]
        device.write_register(0,1) #resetting the device
        for address, value in r.items():
            if isinstance(value, list):
                print(f"0x{i2c_addr:x}[{i}] Writing {len(value)}-value block starting at register 0x{address:X}")
                device.write_block(address, value)
            else:
                print(f"0x{i2c_addr:x}[{i}] Writing value 0x{value:X} to register 0x{address:X}")
                device.write_register(address, value)
            time.sleep(0.1)

0x38[0] Writing value 0x0 to register 0x0
0x38[0] Writing value 0x0 to register 0x1
0x38[0] Writing value 0x0 to register 0x6
0x38[0] Writing value 0x0 to register 0xB
0x38[0] Writing value 0x0 to register 0xC
0x38[0] Writing value 0x0 to register 0xF
0x38[0] Writing value 0x0 to register 0x14
0x38[0] Writing value 0x0 to register 0x15
0x38[0] Writing value 0x0 to register 0x16
0x38[0] Writing value 0x2000003 to register 0x18
0x38[0] Writing value 0x744 to register 0x19
0x38[0] Writing value 0x0 to register 0x1F
0x38[0] Writing value 0x0 to register 0x1E
0x38[0] Writing value 0x0 to register 0x1A
0x38[0] Writing value 0x0 to register 0x1B
0x38[0] Writing 16-value block starting at register 0x20
0x38[0] Writing 2-value block starting at register 0x120
0x38[1] Writing value 0x0 to register 0x0
0x38[1] Writing value 0x0 to register 0x1
0x38[1] Writing value 0x0 to register 0x6
0x38[1] Writing value 0x0 to register 0xB
0x38[1] Writing value 0x0 to register 0xC
0x38[1] Writing value 0x0 to 

In [12]:
# get trigger configuration
trigger_config = ustx_ctrl.get_trigger()
print(json.dumps(trigger_config, indent=4))


{
    "TriggerFrequencyHz": 0,
    "TriggerMode": "UNKNOWN",
    "TriggerPulseCount": 0,
    "TriggerPulseWidthUsec": 0,
    "TriggerStatus": "NOT CONFIGURED"
}


In [11]:
# set trigger configuration
trigger_config = {
    "TriggerFrequencyHz": 50,
    "TriggerMode": 1,
    "TriggerPulseCount": 0,
    "TriggerPulseWidthUsec": 250
}
r = ustx_ctrl.set_trigger(data=trigger_config)
format_and_print_hex(r)

In [13]:
# start trigger pulse
print("Turn Trigger On")
ustx_ctrl.start_trigger()

Turn Trigger On


In [14]:
# stop trigger pulse
print("Turn Trigger Off")
ustx_ctrl.stop_trigger()

Turn Trigger Off


In [21]:
# close teh communication port
ustx_ctrl.uart.close()